In [ ]:
import sys
sys.path.insert(0, '..')

from paus_utils import w_central, z_NB

from jpasLAEs.utils import flux_to_mag, bin_centers

import pickle

import numpy as np

from load_paus_mocks import load_mock_dict

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import model_selection

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 12})

In [ ]:
field_name = 'W3'
savedir = '/home/alberto/almacen/PAUS_data/LF_corrections'

nb_min, nb_max = 0, 16

with open(f'{savedir}/mock_dict_{field_name}_nb{nb_min}-{nb_max}.pkl', 'rb') as f:
    mock_dict = pickle.load(f)

# del mock_dict['SFG']

mock_dict['GAL'].keys()

In [ ]:
# Get the minimum number of candidates to set the set length
N_candidates_list = []
for mock_name, mock in mock_dict.items():
    N_candidates_list.append(sum(mock['nice_lya_0']))

set_len = np.min(N_candidates_list)
print(N_candidates_list)
print(f'{set_len=}')

In [ ]:
# Make the set for each class
tt_set = None
labels = None
rmag = None
zspec = None
zphot = None
L_Arr = None

for mock_name, mock in mock_dict.items():
    mock_len = len(mock['zspec'])
    nice_lya = mock['nice_lya_0']
    np.random.seed(299792458)
    selection = np.random.choice(np.arange(mock_len)[nice_lya], set_len,
                                 replace=False)
    this_set = np.hstack([
        mock['flx'][:40, selection].T * 1e17, # NBs
        mock['lya_NB'][selection].reshape(-1, 1),
        mock['r_mag'][selection].reshape(-1, 1),
        mock['flx'][41:45, selection].T * 1e17, # BBs
    ])

    if tt_set is None:
        tt_set = this_set
        this_rmag = flux_to_mag(mock['flx'][-4, selection], w_central[-4])
        rmag = this_rmag
        zspec = mock['zspec'][selection]
        L_Arr = mock['L_lya'][selection]
        zphot = z_NB(mock['lya_NB'])[selection]
    else:
        tt_set = np.vstack([tt_set, this_set])

        this_rmag = flux_to_mag(mock['flx'][-4, selection], w_central[-4])
        rmag = np.concatenate([rmag, this_rmag])
        zspec = np.concatenate([zspec, mock['zspec'][selection]])
        L_Arr = np.concatenate([L_Arr, mock['L_lya'][selection]])
        zphot = np.concatenate([zphot, z_NB(mock['lya_NB'])[selection]])
    

label_names = []
for i in range(len(mock_dict)):
    mock_name = list(mock_dict.keys())[i]
    print(f'{i} for {mock_name}')
    if labels is None:
        labels = np.ones(set_len).astype(int) * i
    else:
        labels = np.concatenate([labels, np.ones(set_len).astype(int) * i])
    label_names.append(mock_name)
label_names.append('?')

In [ ]:
# Train/Test split
split_seed = 299792458
x_train, x_test, y_train, y_test =\
    model_selection.train_test_split(tt_set, labels, test_size=0.2,
                                     random_state=split_seed)

## Pre-processing ##

x_train[:, :40] /= np.sum(x_train[:, :40], axis=1).reshape(-1, 1)
x_train[:, 41:45] /= np.sum(x_train[:, 41:45], axis=1).reshape(-1, 1)
x_train[:, 40] /= 16.
x_train[:, 41] /= 24.

x_test[:, :40] /= np.sum(x_test[:, :40], axis=1).reshape(-1, 1)
x_test[:, 41:45] /= np.sum(x_test[:, 41:45], axis=1).reshape(-1, 1)
x_test[:, 40] /= 16.
x_test[:, 41] /= 24.

# Standard scaler
# scaler = StandardScaler()
# scaler.fit(x_train)
# x_train = scaler.transform(x_train)
# x_test = scaler.transform(x_test)

# PCA
# pca = PCA(n_components=0.999999, svd_solver='full')

# pca.fit(x_train)
# x_train = pca.transform(x_train)
# x_test = pca.transform(x_test)
print(x_train.shape)

In [ ]:
def do_grid_search(algorithm, search_mode='random'):
    # Create the parameter grid based on the results of random search
    if algorithm == 'nn':
        param_grid = {
            'hidden_layer_sizes': [(60, 60), (60, 60, 60), (60, 40), (50, 50),
                                   (50, 50, 20), (40, 20),],
            'solver': ['adam'],
            'alpha': [1e-3, 1e-4, 1e-5],
            'batch_size': [250, 300, 500, 'auto'],
            'learning_rate': ['adaptive', 'constant'],
        }
        # Create a based model
        model = MLPClassifier()
    elif algorithm == 'rf':
        param_grid = {
            'random_state': [22],
            'n_estimators': [50, 100, 300, 500, 1000],
            'bootstrap': [True, False],
            'max_depth': [20, 50, 70, 100],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        model = RandomForestClassifier()
    else:
        raise Exception('Model not known')

    # Instantiate the grid search model
    if search_mode == 'grid':
        grid_search = GridSearchCV(
            estimator=model, param_grid=param_grid,
            cv=3, n_jobs=-1, pre_dispatch='2*n_jobs',
            verbose=3,
        )
    elif search_mode == 'random':
        grid_search = RandomizedSearchCV(
            estimator=model, param_distributions=param_grid,
            cv=3, n_jobs=-1, pre_dispatch='2*n_jobs',
            verbose=3,
        )
    else:
        raise Exception('What?')

    grid_search.fit(x_train, y_train)

    return grid_search.best_params_

model = 'nn'
search_mode = 'random'
# best_params = do_grid_search(model, search_mode=search_mode)
if model == 'nn':
    best_params = {'alpha': 0.0001, 'batch_size': 300, 'hidden_layer_sizes': (60, 60), 'learning_rate': 'adaptive', 'max_iter': 10000, 'shuffle': False, 'solver': 'adam'} 
elif model == 'rf':
    best_params = {'random_state': 22, 'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 20, 'bootstrap': False}
print(best_params)

In [ ]:
if model == 'nn':
    cl_best = MLPClassifier(**best_params)
elif model == 'rf':
    cl_best = RandomForestClassifier(**best_params)

cl_best.fit(x_train, y_train)
test_score = cl_best.score(x_test, y_test)
train_score = cl_best.score(x_train, y_train)
print(f'Score\n\nTrain: {train_score:0.3f}\nTest: {test_score:0.3f}')

In [ ]:
# Predict test
pred_test = cl_best.predict(x_test)
log_p = cl_best.predict_log_proba(x_test)

class_log_p = log_p[np.arange(len(log_p)), pred_test]
log_p_mask = (class_log_p > np.log(0.9))

pred_test[log_p_mask] = 5

In [ ]:
# import pickle

# # Save the classifier
# save_dir = '/home/alberto/almacen/PAUS_data/ML_classifier'
# with open(f'{save_dir}/source_classifier.sav', 'wb') as file:
#     pickle.dump(cl_best, file)
# with open(f'{save_dir}/source_pca.sav', 'wb') as file:
#     pickle.dump(pca, file)
# with open(f'{save_dir}/source_scaler.sav', 'wb') as file:
    # pickle.dump(scaler, file)

In [ ]:
rmag_train, rmag_test =\
    model_selection.train_test_split(rmag, test_size=0.2, random_state=split_seed)
zspec_train, zspec_test =\
    model_selection.train_test_split(zspec, test_size=0.2, random_state=split_seed)
L_Arr_train, L_Arr_test =\
    model_selection.train_test_split(L_Arr, test_size=0.2, random_state=split_seed)
zphot_train, zphot_test =\
    model_selection.train_test_split(zphot, test_size=0.2, random_state=split_seed)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
r_mask = (rmag_test < 22)
cm = confusion_matrix(y_test[r_mask], pred_test[r_mask])

# Plot confusion matrix
cm = cm.astype('float') / cm.sum(axis=0)[np.newaxis, :]
sns.heatmap(cm, annot=True, cmap="Blues", fmt='.2f',
            xticklabels=label_names, yticklabels=label_names,
            cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title('$r < 22$')
plt.show()

In [ ]:
# Compute confusion matrix
r_mask = (rmag_test > 22)
cm = confusion_matrix(y_test[r_mask], pred_test[r_mask])

# Plot confusion matrix
cm = cm.astype('float') / cm.sum(axis=0)[np.newaxis, :]
sns.heatmap(cm, annot=True, cmap="Blues", fmt='.2f',
            xticklabels=label_names, yticklabels=label_names,
            cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title('$r > 22$')
plt.show()

In [ ]:
# Compute confusion matrix
r_mask = (rmag_test >= 2)
cm = confusion_matrix(y_test[r_mask], pred_test[r_mask])

# Plot confusion matrix
cm = cm.astype('float') / cm.sum(axis=0)[np.newaxis, :]
sns.heatmap(cm, annot=True, cmap="Blues", fmt='.2f',
            xticklabels=label_names, yticklabels=label_names,
            cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title('r $\geq$ 0')
plt.show()

In [ ]:
from jpasLAEs.utils import bin_centers

extra_mask = (rmag_test < 24)
laes_as_laes = ((pred_test == 2) | (pred_test == 3)) & ((y_test == 2) | (y_test == 3)) & extra_mask
laes_as_cont = ((pred_test == 1) | (pred_test == 4)) & ((y_test == 2) | (y_test == 3)) & extra_mask
laes_as_gal = ((pred_test == 4)) & ((y_test == 2) | (y_test == 3)) & extra_mask
laes_as_badqso = (pred_test == 1) & ((y_test == 2) | (y_test == 3)) & extra_mask
badqso_as_badqso = (pred_test == 1) & (y_test == 1)
gal_as_cont = (y_test == 4) & ((pred_test == 4) | (pred_test == 1))
gal_as_lae = (y_test == 4) & ((pred_test == 2) | (pred_test == 3))

z_bins = np.linspace(2.7, 4, 15)
z_bins_c = bin_centers(z_bins)
h_good_laes, _ = np.histogram(zphot_test[laes_as_laes], z_bins)
h_bad_laes, _ = np.histogram(zphot_test[laes_as_cont], z_bins)
h_laes_as_gal, _ = np.histogram(zphot_test[laes_as_gal], z_bins)
h_laes_as_badqso, _ = np.histogram(zphot_test[laes_as_badqso], z_bins)
h_gal_as_cont, _ = np.histogram(zphot_test[gal_as_cont], z_bins)
h_gal_as_lae, _ = np.histogram(zphot_test[gal_as_lae], z_bins)
h_all_gal, _ = np.histogram(zphot_test[pred_test == 4], z_bins)
h_badqso_as_badqso, _ = np.histogram(zphot_test[badqso_as_badqso], z_bins)
h_all_badqso, _ = np.histogram(zphot_test[pred_test == 1], z_bins)


fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(z_bins_c, h_good_laes / (h_good_laes + h_bad_laes), label='LAEs as LAEs')
ax.plot(z_bins_c, h_laes_as_gal / (h_good_laes + h_bad_laes), label='LAEs as GAL')
ax.plot(z_bins_c, h_laes_as_badqso / (h_good_laes + h_bad_laes), label='LAEs as low-z QSO')
ax.plot(z_bins_c, h_gal_as_cont / h_all_gal, label='GAL as GAL')
ax.plot(z_bins_c, h_badqso_as_badqso / h_all_badqso, label='low-z QSO as low-z QSO')


ax.legend(fontsize=11)
ax.set_ylim(0, 1)

ax.set_xlabel('zphot')

plt.show()

In [ ]:
extra_mask = (rmag_test < 24)
laes_as_laes = ((pred_test == 2) | (pred_test == 3)) & ((y_test == 2) | (y_test == 3)) & extra_mask
laes_as_cont = ((pred_test == 1) | (pred_test == 4)) & ((y_test == 2) | (y_test == 3)) & extra_mask
laes_as_gal = ((pred_test == 4)) & ((y_test == 2) | (y_test == 3)) & extra_mask
laes_as_badqso = (pred_test == 1) & ((y_test == 2) | (y_test == 3)) & extra_mask
badqso_as_badqso = (pred_test == 1) & (y_test == 1)
gal_as_cont = (y_test == 4) & ((pred_test == 4) | (pred_test == 1))
gal_as_lae = (y_test == 4) & ((pred_test == 2) | (pred_test == 3))

L_lya_bins = np.linspace(42, 46, 15)
L_lya_bins_c = bin_centers(L_lya_bins)
h_good_laes, _ = np.histogram(L_Arr_test[laes_as_laes], L_lya_bins)
h_bad_laes, _ = np.histogram(L_Arr_test[laes_as_cont], L_lya_bins)
h_laes_as_gal, _ = np.histogram(L_Arr_test[laes_as_gal], L_lya_bins)
h_laes_as_badqso, _ = np.histogram(L_Arr_test[laes_as_badqso], L_lya_bins)
h_gal_as_cont, _ = np.histogram(L_Arr_test[gal_as_cont], L_lya_bins)
h_gal_as_lae, _ = np.histogram(L_Arr_test[gal_as_lae], L_lya_bins)
h_all_gal, _ = np.histogram(L_Arr_test[pred_test == 4], L_lya_bins)
h_badqso_as_badqso, _ = np.histogram(L_Arr_test[badqso_as_badqso], L_lya_bins)
h_all_badqso, _ = np.histogram(L_Arr_test[pred_test == 1], L_lya_bins)


fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(L_lya_bins_c, h_good_laes / (h_good_laes + h_bad_laes), label='LAEs as LAEs')
ax.plot(L_lya_bins_c, h_laes_as_gal / (h_good_laes + h_bad_laes), label='LAEs as GAL')
ax.plot(L_lya_bins_c, h_laes_as_badqso / (h_good_laes + h_bad_laes), label='LAEs as low-z QSO')
ax.plot(L_lya_bins_c, h_gal_as_cont / h_all_gal, label='GAL as GAL')
ax.plot(L_lya_bins_c, h_badqso_as_badqso / h_all_badqso, label='low-z QSO as low-z QSO')


ax.legend(fontsize=11)
ax.set_ylim(0, 1)

ax.set_xlabel('L_lya')

plt.show()